# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
from ou_noise import OUNoise
from utils import hard_update, soft_update
from replay_buffer import ReplayBuffer

import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Actor(nn.Module):
    """Actor (Policy) Model."""
    def __init__(self, 
                 in_actor,
                 hidden_in_actor=256, 
                 hidden_out_actor=256,
                 out_actor=2,
                 leak=0.01, 
                 seed=42):
        """ Initialize parameters and build model.
        Params
        ======
            in_actor (int): Size of state tensor
            hidden_in_actor (int): Number of hidden units in FC layer 1
            hidden_out_actor (int): Number of hidden units in FC layer 2           
            out_actor (int): Size of action tensor
            seed (int): Random seed                    
            leak (float): the leak rate for leaky ReLU, i.e. the alpha in (x < 0) * alpha * x + (x >= 0) * x
        """
        super(Actor, self).__init__()
        self.leak = leak
        self.seed = torch.manual_seed(seed)

        self.fc1 = nn.Linear(in_actor, hidden_in_actor)
        self.fc2 = nn.Linear(hidden_in_actor, hidden_out_actor)
        self.fc3 = nn.Linear(hidden_out_actor, out_actor)

        self.bn = nn.BatchNorm1d(in_actor)
        self.reset_parameters()

    def reset_parameters(self):
        """Initialize FC layers followed by leaky ReLU using Kaiming He's (2015) approach.
        Source: https://arxiv.org/pdf/1502.01852v1.pdf
        For more info see here:
            https://www.jefkine.com/deep/2016/08/08/initialization-of-deep-networks-case-of-rectifiers/
        """
        torch.nn.init.kaiming_normal_(
            self.fc1.weight.data,
            a=self.leak, 
            nonlinearity='leaky_relu',
            mode='fan_in')
        torch.nn.init.kaiming_normal_(
            self.fc2.weight.data,
            a=self.leak,
            nonlinearity='leaky_relu',
            mode='fan_in')
        torch.nn.init.uniform_(self.fc3.weight.data,
                               -3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
#         state = self.bn(state)
        x = F.leaky_relu(self.fc1(state), negative_slope=self.leak)
        x = F.leaky_relu(self.fc2(x), negative_slope=self.leak)
        x = torch.tanh(self.fc3(x))
        return x

In [8]:
class Critic(nn.Module):
    """Critic (Value) Model."""
    def __init__(self,
                 in_critic,
                 hidden_in_critic=256,
                 hidden_out_critic=256,
                 out_critic=1, 
                 leak=0.01, 
                 seed=42):
        """Initialize parameters and build model.
        Params
        ======
            in_critic (int): Dimension of each state            
            hidden_in_critic (int): Number of nodes in the first hidden layer
            hidden_out_critic (int): Number of nodes in the second hidden layer
            out_critic (int): Dimension of each action
            leak (float): Leakiness of leaky ReLU
        """
        super(Critic, self).__init__()
        self.leak = leak
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(in_critic, hidden_in_critic)
        self.fc2 = nn.Linear(hidden_in_critic, hidden_out_critic)
        self.fc3 = nn.Linear(hidden_out_critic, out_critic)
        self.reset_parameters()

    def reset_parameters(self):
        """Initialize FC layers followed by leaky ReLU using Kaiming He's (2015) approach.
        Source: https://arxiv.org/pdf/1502.01852v1.pdf
        For more info see here:
            https://www.jefkine.com/deep/2016/08/08/initialization-of-deep-networks-case-of-rectifiers/
        """
        torch.nn.init.kaiming_normal_(self.fc1.weight.data, a=self.leak,
                                      nonlinearity='leaky_relu', mode='fan_in')
        torch.nn.init.kaiming_normal_(self.fc2.weight.data, a=self.leak,
                                      nonlinearity='leaky_relu', mode='fan_in')
        torch.nn.init.uniform_(self.fc3.weight.data, -3e-3, 3e-3)

    def forward(self, state):
        """ Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        x = F.leaky_relu(self.fc1(state), negative_slope=self.leak)
        x = F.leaky_relu(self.fc2(x), negative_slope=self.leak)
        x = self.fc3(x)
        return x

In [9]:
class DDPGAgent:
    def __init__(self, 
                 in_actor, 
                 hidden_in_actor,
                 hidden_out_actor,
                 out_actor, 
                 in_critic,
                 hidden_in_critic,
                 hidden_out_critic,
                 out_critic,
                 device,
                 relu_leak=1e-2,
                 seed=42,
                 lr_actor=1.0e-2, 
                 lr_critic=1.0e-2,
                 noise_mul=0.1):
        super(DDPGAgent, self).__init__()
        self.device=device
        self.actor = Actor(
            in_actor,
            hidden_in_actor, 
            hidden_out_actor, 
            out_actor, 
            relu_leak, 
            seed).to(device)
        self.critic = Critic(
            in_critic,
            hidden_in_critic,
            hidden_out_critic, 
            out_critic, 
            relu_leak, 
            seed).to(device)
        self.target_actor = Actor(
            in_actor, 
            hidden_in_actor,
            hidden_out_actor,
            out_actor, 
            relu_leak,
            seed).to(device)
        self.target_critic = Critic(
            in_critic, 
            hidden_in_critic,
            hidden_out_critic,
            out_critic, 
            relu_leak,
            seed).to(device)

        self.noise = OUNoise(out_actor, seed=seed, scale=1.0)
        
        # initialize targets same as original networks
        hard_update(self.target_actor, self.actor)
        hard_update(self.target_critic, self.critic)

        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=lr_critic, weight_decay=1.e-5)   
        
    def act(self, states, noise_mul=0.1):
        self.actor.eval()
        with torch.no_grad():
            actions = self.actor(states).cpu().data.numpy()
        actions += self.noise.sample() * noise_mul
        return np.clip(actions, -1, 1)        
    
    def target_act(self, states, noise_mul=0.1):
        self.target_actor.eval()
        with torch.no_grad():
            actions = self.target_actor(states).cpu().data.numpy()
        actions += self.noise.sample() * noise_mul
        return np.clip(actions, -1, 1)
    
    def reset(self):
        self.noise.reset()

In [85]:
class MADDPGAgent:
    def __init__(
        self,
        num_agents,
        in_actor, 
        hidden_in_actor,
        hidden_out_actor,
        out_actor, 
        hidden_in_critic,
        hidden_out_critic,
        out_critic,
        device,
        relu_leak=1e-2,
        seed=42,
        lr_actor=1.0e-2, 
        lr_critic=1.0e-2,
        replay_buffer_size=int(1e6),
        batch_size=64,
        gamma=0.99,
        tau=1e-3,
        update_every=1,
        num_updates=1
    ):        
        in_critic = in_actor * num_agents + out_actor * 2
        self.agents = []
        self.num_agents = num_agents
        self.out_actor = out_actor
        self.replay_buffer = ReplayBuffer(num_agents * out_actor, replay_buffer_size, batch_size, seed, device)
        self.batch_size = batch_size
        self.gamma = gamma
        self.tau = tau
        self.device = device
        self.steps = 0
        self.update_every = update_every
        self.num_updates = num_updates
        self.noise_scale = 1.0
        for _ in range(num_agents):
            self.agents.append(
                DDPGAgent(
                    in_actor, hidden_in_actor, hidden_out_actor, out_actor,
                    in_critic, hidden_in_critic, hidden_out_critic, out_critic, 
                    device, relu_leak, seed, lr_actor, lr_critic)
            )          

    def act(self, states, noise_mul=1.0):
        """Returns actions for given state as per current policy."""
        actions = np.zeros((self.num_agents, self.out_actor))
        for agent_idx, state in enumerate(states):
            state = torch.from_numpy(state).float().to(self.device)
            if state.dim() < 2:
                state = state.unsqueeze(0)
            curr_agent = self.agents[agent_idx]
            action = curr_agent.act(state, noise_mul)
            actions[agent_idx, :] = action
        return actions
            
    def step(self, states, actions, rewards, next_states, dones):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        self.steps += 1
        self.replay_buffer.add(states, actions, rewards, next_states, dones)

        if (len(self.replay_buffer) > self.batch_size) and (self.steps % self.update_every == 0):
            # reset steps so we don't overflow at some point
            self.steps = 0
#             print("learning")
            for _ in range(self.num_updates):
                experiences = self.replay_buffer.sample()
                self.learn(experiences, self.gamma)
  
    def learn(self, experiences, gamma):
        
        obs_full, actions_full, rewards_full, next_obs_full, dones_full = experiences
        print("obs_full.shape={}".format(obs_full.shape))
        print("stacked_obs_full.shape={}\n".format(obs_full.shape))
        
        for agent_idx, agent in enumerate(self.agents):
            
            agent.critic_optimizer.zero_grad()
            
            agent_torch_idx = torch.tensor([agent_idx]).to(self.device)
            other_agent_idx = (agent_idx + 1) % 2                  
            other_agent_torch_idx = torch.tensor([other_agent_idx]).to(self.device)
            
            # Select for current agent
            obs = torch.index_select(obs_full, 1, agent_torch_idx).squeeze()
            actions = torch.index_select(actions_full, 1, agent_torch_idx)
            rewards = torch.index_select(rewards_full, 1, agent_torch_idx)
            next_obs = torch.index_select(next_obs_full, 1, agent_torch_idx).squeeze()
            dones = torch.index_select(dones_full, 1, agent_torch_idx)            
            
            target_actions = agent.target_act(next_obs)
            target_actions = target_actions.squeeze()
            
#             actions = torch.from_numpy(actions).to(self.device)
            target_actions = torch.from_numpy(target_actions).to(self.device)
            
            print("type(target_actions)={}".format(type(target_actions)))
            print("net_obs.shape={}".format(next_obs.shape))
            print("target_actions.shape={}".format(target_actions.shape))
            target_critic_input = torch.cat((
                next_obs_full.reshape(self.batch_size, -1), 
                target_actions), dim=1).to(device)
            print("target_critic_input.shape={}".format(target_critic_input.shape))

        
#         target_critic_input = torch.cat((next_obs_full.t(),target_actions), dim=1).to(device)
        
#         with torch.no_grad():
#             q_next = agent.target_critic(target_critic_input)
        
#         y = reward[agent_number].view(-1, 1) + self.discount_factor * q_next * (1 - done[agent_number].view(-1, 1))
#         action = torch.cat(action, dim=1)
#         critic_input = torch.cat((obs_full.t(), action), dim=1).to(device)
#         q = agent.critic(critic_input)

#         huber_loss = torch.nn.SmoothL1Loss()
#         critic_loss = huber_loss(q, y.detach())
#         critic_loss.backward()
#         #torch.nn.utils.clip_grad_norm_(agent.critic.parameters(), 0.5)
#         agent.critic_optimizer.step()

#         #update actor network using policy gradient
#         agent.actor_optimizer.zero_grad()
#         # make input to agent
#         # detach the other agents to save computation
#         # saves some time for computing derivative
#         q_input = [ self.maddpg_agent[i].actor(ob) if i == agent_number \
#                    else self.maddpg_agent[i].actor(ob).detach()
#                    for i, ob in enumerate(obs) ]
                
#         q_input = torch.cat(q_input, dim=1)
#         # combine all the actions and observations for input to critic
#         # many of the obs are redundant, and obs[1] contains all useful information already
#         q_input2 = torch.cat((obs_full.t(), q_input), dim=1)
        
#         # get the policy gradient
#         actor_loss = -agent.critic(q_input2).mean()
#         actor_loss.backward()
#         #torch.nn.utils.clip_grad_norm_(agent.actor.parameters(),0.5)
#         agent.actor_optimizer.step()

#         al = actor_loss.cpu().detach().item()
#         cl = critic_loss.cpu().detach().item()
#         logger.add_scalars('agent%i/losses' % agent_number,
#                            {'critic loss': cl,
#                             'actor_loss': al},
#                            self.iter)
        
        
        
    def reset(self):
        self.noise.reset()
        
    def save_agents(self):
        raise NotImplementedError("Saving is not implemented yet.")


In [86]:
num_agents = 2

in_actor = 24
hidden_in_actor = 128
hidden_out_actor = 64
out_actor = 2

hidden_in_critic = 128
hidden_out_critic = 64

out_critic = 1

lr_actor = 1e-2
lr_critic = 1e-2
relu_leak = 1e-2
seed = 42

gpu_id = 0
use_gpu = True
device = 'cuda:{}'.format(gpu_id) if use_gpu else 'cpu'

batch_size = 64
replay_buffer_size = int(1e6)
gamma = 0.99
tau = 1e-3
update_every = 1
num_updates = 1

meta_agent = MADDPGAgent(  
    num_agents,
    in_actor, 
    hidden_in_actor,
    hidden_out_actor,
    out_actor, 
    hidden_in_critic,
    hidden_out_critic,
    out_critic,
    device,
    relu_leak,
    seed,
    lr_actor, 
    lr_critic,
    replay_buffer_size,
    batch_size,
    gamma,
    tau,
    update_every,
    num_updates)

In [87]:
env_info = env.reset(train_mode=False)[brain_name]

states = env_info.vector_observations

actions = meta_agent.act(states)
print(actions)

[[-0.02232587  0.04771298]
 [ 0.02163291  0.40717682]]


In [88]:
num_episodes = 20

num_steps_max = 1000

print_every = 10

all_scores = np.zeros(num_agents)
ctr = 0

for i in range(num_episodes):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)

    for j in range(num_steps_max):
        
        ctr += 1
        
        actions = meta_agent.act(states)
#         print(actions)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards        
        dones = env_info.local_done
#         print("states.shape={}".format(states.shape))
#         print("actions.shape={}".format(actions.shape))
#         print("rewards.shape={}".format(len(rewards)))
#         print("next_states.shape={}".format(next_states.shape))
#         print("dones.shape={}".format(len(dones)))
        actions = torch.from_numpy(actions).to(meta_agent.device)
        meta_agent.step(states, actions, rewards, next_states, dones)
        scores += env_info.rewards
        
        all_scores += env_info.rewards
        
        states = next_states
        if np.any(dones): 
            print("episode {} done after {} steps".format(i, j))
            break
            
        if j > batch_size and j % print_every == 0:
            print("episode {}, step {}: done".format(i, j))
    if i % print_every == 0:        
        print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))
        print("Average score: {}".format(np.mean(all_scores) / ctr))

episode 0 done after 13 steps
Score (max over agents) from episode 0: 0.0
Average score: -0.0003571428491600922
episode 1 done after 13 steps
episode 2 done after 13 steps
episode 3 done after 13 steps
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
tar

obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([6

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([

obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([6

obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([6

...

obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([6

obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([6

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([

obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([6

obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([64, 24])
target_actions.shape=torch.Size([64, 2])
target_critic_input.shape=torch.Size([64, 50])
obs_full.shape=torch.Size([64, 2, 24])
stacked_obs_full.shape=torch.Size([64, 2, 24])

type(target_actions)=<class 'torch.Tensor'>
net_obs.shape=torch.Size([6